<a href="https://colab.research.google.com/github/periclesrialto/IndustrialAutomation/blob/main/AlarmMatrix_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [264]:
import pandas as pd
import math

In [265]:
df = pd.read_csv('Alarm_matrix.csv', header = None, sep=';')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,NaN,NaN,NaN,Alarm Area:,Startup,NaN,NaN,NaN,Drying Under 180,NaN,...,NaN,NaN,Transport,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Item,Component Code,Alarm Text,Alarm (ctrl+c ctrl+v),ALL,EMERGENCY,END,INTERRUPT,ALL,EMERGENCY,...,END,INTERRUPT,ALL,EMERGENCY,END,INTERRUPT,NaN,NaN,NaN,NaN
2,001,XYZ,TEXTO,AM001 - XYZ - TEXTO,EMERGENCY,1,0,0,INTERRUPT,0,...,0,0,EMERGENCY,1,0,0,NaN,NaN,NaN,NaN
3,002,XYZ,TEXTO,AM002 - XYZ - TEXTO,INTERRUPT,0,0,1,INTERRUPT,0,...,0,0,EMERGENCY,1,0,0,NaN,NaN,NaN,NaN
4,003,XYZ,TEXTO,AM003 - XYZ - TEXTO,END,0,1,0,INTERRUPT,0,...,0,0,EMERGENCY,1,0,0,NaN,NaN,NaN,NaN


In [266]:
var_base_len = 32
alarm_category = []
for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    shift = 1
    while df[column + shift][1] != 'ALL':
      alarm_category.append(df[column+ shift][1])
      shift += 1
    break
for category in range(len(alarm_category)):
  alarm_category[category] = alarm_category[category].lower()
  alarm_category[category] = alarm_category[category].capitalize()
  alarm_category[category] += 'Group'

machine_state = []
for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    machine_state.append(df[column][0])
for state in range(len(machine_state)):
  machine_state[state] = machine_state[state].replace(" ", "")

column_names = []
for state in range(len(machine_state)):
  for category in range(len(alarm_category)):
    column_names.append(machine_state[state] + '.' + alarm_category[category])

dwords_df = pd.DataFrame(columns=column_names)

for empty in range(math.ceil(len(df.index)/var_base_len)):
  dwords_df = dwords_df.append(pd.Series(dtype='object'), ignore_index=True)

In [267]:
dwords_range = (math.ceil(len(df.index)/var_base_len))*len(alarm_category)*len(machine_state)
dwords_list = ["" for x in range(dwords_range)]

pointer = 0
row = 0

for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    for shift in range(1,4):
      for index in range(2,len(df.index)):
        dwords_list[pointer] += str(df[column+shift][index])
        row += 1
        if row==32 and pointer < len(dwords_list):
          pointer += 1
          row = 0
        if index == (len(df.index)-1):
          for complete in range(var_base_len-row):
            dwords_list[pointer] += str(0)
          if pointer < len(dwords_list):
            pointer += 1
            row = 0

for invert in range(len(dwords_list)):
  dwords_list[invert] += '#2'
  dwords_list[invert] = dwords_list[invert][::-1]

In [268]:
pointer = 0
for column in range(len(dwords_df.columns)):
  for index in range(len(dwords_df.index)):
    dwords_df.at[index,column_names[column]]=dwords_list[pointer]
    pointer += 1

In [269]:
print('Para que o script em SCL funcione:\n')

print('Crie um array de alarmes:')
print(' --> DB_ALM_AREA como Array[0..'+str(len(dwords_df.index)-1)+'] of Dword\n')

print('Crie um datatype que contenha:')
print(' --> EmergencyGroup como Dword')
print(' --> EndGroup como Dword')
print(' --> InterruptGroup como Dword')
print(' --> EmergencyOut como Bool')
print(' --> EndOut como Bool')
print(' --> InterruptOut como Bool')

print('\nCrie instâncias do datatype com os nomes:')
for state in range(len(machine_state)):
    print(' --> '+str(machine_state[state]))

print('\nCopie o script abaixo para o TIA Portal:\n')
category_pointer = 0
for column in range(len(dwords_df.columns)): 
  if category_pointer == 0:
    print('REGION Masks '+str(dwords_df.columns[column].split('.')[0])+'\n')
  print(' REGION Comparing Alarm Matrix with '+str(dwords_df.columns[column])+'\n')
  for alarm_dw in range(len(dwords_df.index)):
    print('  IF (DB_ALM_AREA['+str(alarm_dw)+'] AND ('+str(dwords_df[dwords_df.columns[column]][alarm_dw])+')) <> 0 THEN')
    print('   '+str(".".join(dwords_df.columns[column].split('.')))+'.%X'+str(alarm_dw)+' := TRUE;')
    print('  ELSE')
    print('   '+str(".".join(dwords_df.columns[column].split('.')))+'.%X'+str(alarm_dw)+' := FALSE;')
    print('  END_IF;\n')
    category_pointer += 1
  print(' END_REGION\n')
  print(' REGION Compiles the '+str(" ".join(dwords_df.columns[column].split('.')))+' mask output\n')
  print('  IF '+str(".".join(dwords_df.columns[column].split('.')))+' <> 0 THEN')
  print('   '+str(dwords_df.columns[column]).replace('Group','Out')+' := TRUE;')
  print('  ELSE')
  print('   '+str(dwords_df.columns[column]).replace('Group','Out')+' := FALSE;')
  print('  END_IF;\n')
  print(' END_REGION\n')
  if category_pointer == (len(dwords_df.index)*len(alarm_category)):
    print('END_REGION\n')
    category_pointer = 0

Para que o script em SCL funcione:

Crie um array de alarmes:
 --> DB_ALM_AREA como Array[0..9] of Dword

Crie um datatype que contenha:
 --> EmergencyGroup como Dword
 --> EndGroup como Dword
 --> InterruptGroup como Dword
 --> EmergencyOut como Bool
 --> EndOut como Bool
 --> InterruptOut como Bool

Crie instâncias do datatype com os nomes:
 --> Startup
 --> DryingUnder180
 --> DryingOver180
 --> Roasting
 --> Cooling
 --> Transport

Copie o script abaixo para o TIA Portal:

REGION Masks Startup

 REGION Comparing Alarm Matrix with Startup.EmergencyGroup

  IF (DB_ALM_AREA[0] AND (2#11000000000000000000000101010001)) <> 0 THEN
   Startup.EmergencyGroup.%X0 := TRUE;
  ELSE
   Startup.EmergencyGroup.%X0 := FALSE;
  END_IF;

  IF (DB_ALM_AREA[1] AND (2#10000000000000000000000000000000)) <> 0 THEN
   Startup.EmergencyGroup.%X1 := TRUE;
  ELSE
   Startup.EmergencyGroup.%X1 := FALSE;
  END_IF;

  IF (DB_ALM_AREA[2] AND (2#10000000000000000000000000000001)) <> 0 THEN
   Startup.EmergencyGro